# Datasets Validation using  RandomForestClassifier Classifier:

# For the ArticularyWordRecognition Dataset:

In [1]:
from sktime.datasets import load_UCR_UEA_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from pyts.datasets import load_basic_motions
from sklearn.model_selection import train_test_split

/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_series/_check.py:43: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_hierarchical/_check.py:50: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_hierarchical/_check.py:51: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_MULTIINDEX_TYPES = (pd.Int64Index, pd.RangeIndex)
/Users/dinsind/.local/lib/pyth

In [2]:
X,y= load_UCR_UEA_dataset('ArticularyWordRecognition', split="train", return_X_y=True)

In [3]:
X_test,y_test= load_UCR_UEA_dataset('ArticularyWordRecognition', split="test", return_X_y=True)

In [4]:
X_3d = from_nested_to_3d_numpy(X)
X_3d.shape

(275, 9, 144)

In [5]:
X_test_3d= from_nested_to_3d_numpy(X_test)
X_test_3d.shape

(300, 9, 144)

In [6]:
class reservoir_features:
    '''
    creates an object associated with a multivariate dataset
    
    
    '''
    def __init__(self,data,num_features):
        '''
        data: Takes in a multidimensional array (x * y * z) - z>y
        Initializes it
        x: Timeseries
        y: Attributes for a given timeseries observation
        z: timestamped observations (features)
        
        num_features: you must specify the dimension you want to reduce it to
        
        '''
        self.features = []
        self.filters_used = []
        self.original_data = data.copy()
        self.data = data.copy()
        self.num_features = num_features
        self.x = data.shape[0]
        self.y = data.shape[1]
        self.z = data.shape[2]
        # perform checks 
        #1. 3d numpy array
        #2. Each time series should have same number of observations
        #3. num_features should be less than timestamped observations
    
    def normalize(self):
        '''
        Each attribute could potentially be on a different scale
        modifies the original data and performs a min max normalization
        '''
        for i in range(self.original_data.shape[0]):
            for j in range(self.original_data.shape[1]):
                self.data[i][j] = (self.original_data[i][j] - self.original_data[i][j].min())/(self.original_data[i][j].max()-self.original_data[i][j].min())
    
    
    def filters(self,stride_len = [1], num_filters = 1):
        '''
        stride_len: num of columns to skip after each filter multiplication
        num_filters: you can specify the number of filters you need; each filter will be of a differnt size
        size of filter = n*m 
        (n = # of rows = attribute size, 
        m = # of columns)
        '''
        #Have error check to make sure stride len is a list and value is <length of attributes
        n = self.y
        
        
        #Edge case vals is empty/smaller than num_filters
        
        for iteration in range(num_filters):
            m = self._get_m(stride_len[iteration])
            filter_a = np.random.random((n,m))
            print("filter of size ", str(n), "*", str(m), "was created\n")
            self.filters_used.append(filter_a)
            
            temp_features =[]
            for i in range(self.x):
                temp = []
                j = 0
                while j + m < self.data.shape[2]:
                    temp.append((filter_a*self.data[i,:,j:j+m]).mean())
                    j+=stride_len[iteration]
                temp_features.append(temp)
            self.features.append(temp_features)
    
    
    def _get_m(self,stride_len):
        '''
        stride_len: 
        based on stride length,& num_features, we calculate possible filter size 
        '''
        m = self.z -(self.num_features)*stride_len
        return m
    
    
    def result_features(self):
        '''
        if multiple filters were added, takes the average result
        '''
        ans =[]
        for timeseries in range(len(self.features[0])):
            temp =[]
            for feature in range(len(self.features[0][0])):
                val = np.mean([self.features[filter][timeseries][feature] for filter in range(len(self.features))])
                temp.append(val)
            ans.append(temp)
        return ans
        
        

In [7]:
def transform_data(data, num_features, stride_len, num_filters):
    data_transformed = reservoir_features(data ,num_features = num_features)
    #normalize
    data_transformed.normalize()
    #create 2 filters
    data_transformed.filters(stride_len = stride_len, num_filters = num_filters)
    data_transformed = data_transformed.result_features()
    return data_transformed

In [8]:
X_train_transformed= transform_data(X_3d, 40, [3], 1)


filter of size  9 * 24 was created



In [9]:
X_test_transformed=transform_data(X_test_3d, 40, [3], 1)

filter of size  9 * 24 was created



In [10]:
clf = RandomForestClassifier(n_estimators=100,random_state=0)
clf.fit(X_train_transformed, y)



RandomForestClassifier(random_state=0)

In [11]:
clf.score(X_train_transformed, y)

1.0

In [12]:
clf.apply(X_train_transformed)

array([[133,  46,   5, ...,  34, 113,   4],
       [ 54,  19,   5, ...,  25,  15,   4],
       [133,  46,   5, ...,  34, 113,   4],
       ...,
       [ 89, 102, 105, ...,  87,  80, 130],
       [112, 115, 116, ..., 156, 142, 147],
       [ 91, 102, 105, ...,  76,  78, 130]])

In [13]:
clf.predict_log_proba(X_train_transformed)

/Users/dinsind/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:722: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


array([[-0.03045921,        -inf,        -inf, ...,        -inf,
               -inf,        -inf],
       [-0.38566248,        -inf,        -inf, ..., -3.5065579 ,
        -3.5065579 ,        -inf],
       [-0.03045921,        -inf,        -inf, ...,        -inf,
               -inf,        -inf],
       ...,
       [       -inf,        -inf,        -inf, ..., -4.60517019,
               -inf, -2.40794561],
       [       -inf, -3.91202301,        -inf, ...,        -inf,
               -inf, -1.51412773],
       [       -inf,        -inf,        -inf, ...,        -inf,
               -inf, -2.65926004]])

## Accuracy for Train Data:

In [14]:
# Accuracy:
clf_train_pred=clf.predict(X_train_transformed)
clf_acc_score = accuracy_score(y, clf_train_pred)
print(clf_acc_score)


1.0


In [15]:
roc_auc_score(y, clf.predict_proba(X_train_transformed), multi_class='ovr')

1.0

In [16]:
train_f1 = f1_score(y, clf_train_pred, average='weighted')
train_f2 = f1_score(y, clf_train_pred, average='macro')
train_f3 = f1_score(y, clf_train_pred, average='micro')
print(train_f1)
print(train_f2)
print(train_f3)

1.0
1.0
1.0


## For the Test set:

In [17]:

clf_test_pred=clf.predict(X_test_transformed)
clf_test_acc_score = accuracy_score(y_test, clf_test_pred)
print(clf_test_acc_score)

0.44


In [18]:
roc_auc_score(y_test, clf.predict_proba(X_test_transformed), multi_class='ovr')

0.9253587962962961

In [19]:
#f1-score:
test_f1 = f1_score(y_test, clf_test_pred, average='weighted')
test_f2 = f1_score(y_test, clf_test_pred, average='macro')
test_f3 = f1_score(y_test, clf_test_pred, average='micro')
print(test_f1)
print(test_f2)
print(test_f3)

0.4174927888342895
0.4174927888342896
0.44


# For the AtrialFibrillation dataset:

In [20]:
X_AF,y_AF= load_UCR_UEA_dataset('AtrialFibrillation', split="train", return_X_y=True)
X_AF_test,y_AF_test= load_UCR_UEA_dataset('AtrialFibrillation', split="test", return_X_y=True)
X_AF_3d = from_nested_to_3d_numpy(X_AF)
X_AF_3d.shape
X_AF_test_3d= from_nested_to_3d_numpy(X_AF_test)
X_AF_test_3d.shape

print(X_AF_3d.shape)
print(X_AF_test_3d.shape)

(15, 2, 640)
(15, 2, 640)


In [21]:
X_train_transformed_AF = transform_data(X_AF_3d, 40, [3], 1)
X_test_transformed_AF = transform_data(X_AF_test_3d, 40, [3], 1)

filter of size  2 * 520 was created

filter of size  2 * 520 was created



In [22]:
print(len(X_train_transformed_AF))
print(len(X_train_transformed_AF[0]))

15
40


In [23]:


clf_AF= RandomForestClassifier(n_estimators=100,random_state=0)
clf_AF.fit(X_train_transformed_AF, y_AF)

RandomForestClassifier(random_state=0)

In [24]:
clf_AF.score(X_train_transformed_AF, y_AF)

1.0

In [25]:
clf_AF.apply(X_train_transformed_AF)

array([[ 5,  7, 10, ...,  8, 10,  7],
       [ 1,  3,  4, ...,  1,  3,  3],
       [ 1,  3,  3, ...,  1,  3,  3],
       ...,
       [ 7, 10, 11, ...,  9, 10, 10],
       [ 8, 12, 12, ..., 10, 12, 12],
       [ 3,  5,  5, ...,  3,  6,  5]])

## Accuracy for Train Data:


In [26]:
# Accuracy:
clf_train_pred_AF=clf_AF.predict(X_train_transformed_AF)
clf_acc_score_AF = accuracy_score(y_AF, clf_train_pred_AF)
print("Accuracy_score:",clf_acc_score_AF)

train_AF_f1 = f1_score(y_AF, clf_train_pred_AF, average='weighted')
train_AF_f2 = f1_score(y_AF, clf_train_pred_AF, average='macro')
train_AF_f3 = f1_score(y_AF, clf_train_pred_AF, average='micro')
print("f1_score:Weighted:",train_f1)
print("f1_score:macro:",train_f2)
print("f1_score:micro:",train_f3)


Accuracy_score: 1.0
f1_score:Weighted: 1.0
f1_score:macro: 1.0
f1_score:micro: 1.0


In [27]:
roc_auc_score(y_AF, clf_AF.predict_proba(X_train_transformed_AF), multi_class='ovr')

1.0

## Accuracy for Test Data:

In [28]:
# Accuracy:
clf_test_pred_AF=clf_AF.predict(X_test_transformed_AF)
clf_acc_score_test_AF = accuracy_score(y_AF_test, clf_test_pred_AF)
print("Accuracy_score:",clf_acc_score_test_AF)

# F1-Score:
test_AF_f1 = f1_score(y_AF_test, clf_test_pred_AF, average='weighted')
test_AF_f2 = f1_score(y_AF_test, clf_test_pred_AF, average='macro')
test_AF_f3 = f1_score(y_AF_test, clf_test_pred_AF, average='micro')
print("f1_score:Weighted:",test_AF_f1)
print("f1_score:macro:",test_AF_f2)
print("f1_score:micro:",test_AF_f3)

Accuracy_score: 0.26666666666666666
f1_score:Weighted: 0.25071225071225073
f1_score:macro: 0.25071225071225073
f1_score:micro: 0.26666666666666666


In [29]:
roc_auc_score(y_AF_test, clf_AF.predict_proba(X_test_transformed_AF), multi_class='ovr')

0.4066666666666667

# For the Cricket dataset

In [30]:
X_C,y_C= load_UCR_UEA_dataset('Cricket', split="train", return_X_y=True)
X_C_test,y_C_test= load_UCR_UEA_dataset('Cricket', split="test", return_X_y=True)
X_C_3d = from_nested_to_3d_numpy(X_C)
X_C_3d.shape
X_C_test_3d= from_nested_to_3d_numpy(X_C_test)
X_C_test_3d.shape

print(X_C_3d.shape)
print(X_C_test_3d.shape)

(108, 6, 1197)
(72, 6, 1197)


In [31]:
X_train_transformed_C = transform_data(X_C_3d, 200, [3], 1)
X_test_transformed_C = transform_data(X_C_test_3d, 200, [3], 1)

filter of size  6 * 597 was created

filter of size  6 * 597 was created



In [32]:


clf_C= RandomForestClassifier(n_estimators=100,random_state=0)
clf_C.fit(X_train_transformed_C, y_C)

RandomForestClassifier(random_state=0)

In [33]:
clf_C.score(X_train_transformed_C, y_C)


1.0

In [34]:
clf_C.apply(X_train_transformed_C)

array([[34, 42, 31, ..., 25, 24, 44],
       [15, 24, 17, ..., 15, 15, 25],
       [32, 42, 29, ..., 21, 22, 26],
       ...,
       [23, 23, 30, ..., 15, 16, 25],
       [ 9, 16, 18, ..., 15, 14,  6],
       [30, 28, 27, ..., 21, 19, 25]])

# Accuracy for train Data:

In [35]:
# Accuracy:
clf_train_pred_C=clf_C.predict(X_train_transformed_C)
clf_acc_score_C = accuracy_score(y_C, clf_train_pred_C)
print("Accuracy_score:",clf_acc_score_C)

train_C_f1 = f1_score(y_C, clf_train_pred_C, average='weighted')
train_C_f2 = f1_score(y_C, clf_train_pred_C, average='macro')
train_C_f3 = f1_score(y_C, clf_train_pred_C, average='micro')
print("f1_score:Weighted:",train_C_f1)
print("f1_score:macro:",train_C_f2)
print("f1_score:micro:",train_C_f3)

Accuracy_score: 1.0
f1_score:Weighted: 1.0
f1_score:macro: 1.0
f1_score:micro: 1.0


In [36]:
roc_auc_score(y_C, clf_C.predict_proba(X_train_transformed_C), multi_class='ovr')

1.0

# Accuracy for test Data:

In [37]:
# Accuracy:
clf_test_pred_C=clf_C.predict(X_test_transformed_C)
clf_acc_score_test_C = accuracy_score(y_C_test, clf_test_pred_C)
print("Accuracy_score:",clf_acc_score_test_C)

# F1-Score:
test_C_f1 = f1_score(y_C_test, clf_test_pred_C, average='weighted')
test_C_f2 = f1_score(y_C_test, clf_test_pred_C, average='macro')
test_C_f3 = f1_score(y_C_test, clf_test_pred_C, average='micro')
print("f1_score:Weighted:",test_C_f1)
print("f1_score:macro:",test_C_f2)
print("f1_score:micro:",test_C_f3)

Accuracy_score: 0.5
f1_score:Weighted: 0.4862866708454944
f1_score:macro: 0.48628667084549443
f1_score:micro: 0.5


In [38]:
roc_auc_score(y_C_test, clf_C.predict_proba(X_test_transformed_C), multi_class='ovr')

0.8937289562289562

# For the Epilepsy Data:

In [39]:
X_E,y_E= load_UCR_UEA_dataset('Epilepsy', split="train", return_X_y=True)
X_E_test,y_E_test= load_UCR_UEA_dataset('Epilepsy', split="test", return_X_y=True)
X_E_3d = from_nested_to_3d_numpy(X_E)
X_E_3d.shape
X_E_test_3d= from_nested_to_3d_numpy(X_E_test)
X_E_test_3d.shape

print(X_E_3d.shape)
print(X_E_test_3d.shape)

(137, 3, 206)
(138, 3, 206)


In [40]:
X_train_transformed_E = transform_data(X_E_3d, 16, [3], 1)
X_test_transformed_E = transform_data(X_E_test_3d, 16, [3], 1)

filter of size  3 * 158 was created

filter of size  3 * 158 was created



In [41]:

clf_E= RandomForestClassifier(n_estimators=100,random_state=0)
clf_E.fit(X_train_transformed_E, y_E)

RandomForestClassifier(random_state=0)

In [42]:
clf_E.score(X_train_transformed_E, y_E)


1.0

In [43]:
clf_E.apply(X_train_transformed_E)

array([[75, 77, 74, ..., 70, 72, 86],
       [10,  5, 13, ..., 19,  5, 17],
       [18, 16, 24, ..., 19, 19, 21],
       ...,
       [ 3,  1, 11, ...,  9,  4, 10],
       [47, 33, 46, ..., 46, 39, 60],
       [47, 50, 60, ..., 64, 53, 60]])

# Accuracy for Train Data:

In [44]:
# Accuracy:
clf_train_pred_E=clf_E.predict(X_train_transformed_E)
clf_acc_score_E = accuracy_score(y_E, clf_train_pred_E)
print("Accuracy_score:",clf_acc_score_E)

train_E_f1 = f1_score(y_E, clf_train_pred_E, average='weighted')
train_E_f2 = f1_score(y_E, clf_train_pred_E, average='macro')
train_E_f3 = f1_score(y_E, clf_train_pred_E, average='micro')
print("f1_score:Weighted:",train_E_f1)
print("f1_score:macro:",train_E_f2)
print("f1_score:micro:",train_E_f3)

Accuracy_score: 1.0
f1_score:Weighted: 1.0
f1_score:macro: 1.0
f1_score:micro: 1.0


In [45]:
roc_auc_score(y_E, clf_E.predict_proba(X_train_transformed_E), multi_class='ovr')

1.0

# Accuracy for Test Data:


In [46]:
# Accuracy:
clf_test_pred_E=clf_E.predict(X_test_transformed_E)
clf_acc_score_test_E = accuracy_score(y_E_test, clf_test_pred_E)
print("Accuracy_score:",clf_acc_score_test_E)

# F1-Score:
test_E_f1 = f1_score(y_E_test, clf_test_pred_E, average='weighted')
test_E_f2 = f1_score(y_E_test, clf_test_pred_E, average='macro')
test_E_f3 = f1_score(y_E_test, clf_test_pred_E, average='micro')
print("f1_score:Weighted:",test_E_f1)
print("f1_score:macro:",test_E_f2)
print("f1_score:micro:",test_E_f3)

Accuracy_score: 0.3333333333333333
f1_score:Weighted: 0.3312358186822363
f1_score:macro: 0.332110143377749
f1_score:micro: 0.3333333333333333


In [47]:
roc_auc_score(y_E_test, clf_E.predict_proba(X_test_transformed_E), multi_class='ovr')

0.5946577831051114

# For the FingerMovements Data:


In [48]:
X_F,y_F= load_UCR_UEA_dataset('FingerMovements',return_X_y=True)
X_F_Train,X_F_test,y_F_Train,y_F_test= train_test_split(X_F,y_F, test_size=0.5, random_state=0)
X_F_3d = from_nested_to_3d_numpy(X_F_Train)
X_F_3d.shape
X_F_test_3d= from_nested_to_3d_numpy(X_F_test)
X_F_test_3d.shape

print(X_F_3d.shape)
print(X_F_test_3d.shape)

(208, 28, 50)
(208, 28, 50)


In [49]:
X_train_transformed_F = transform_data(X_F_3d, 16, [3], 1)
X_test_transformed_F = transform_data(X_F_test_3d, 16, [3], 1)

filter of size  28 * 2 was created

filter of size  28 * 2 was created



In [50]:

clf_F= RandomForestClassifier(n_estimators=100,random_state=0)
clf_F.fit(X_train_transformed_F, y_F_Train)

RandomForestClassifier(random_state=0)

In [51]:
clf_F.score(X_train_transformed_F, y_F_Train)

1.0

In [52]:
clf_F.apply(X_train_transformed_F)

array([[ 1, 13,  6, ...,  1,  4, 13],
       [50, 68, 83, ..., 61, 65, 66],
       [48, 65, 92, ..., 59, 60, 75],
       ...,
       [36, 65, 64, ..., 30, 60, 33],
       [19, 42, 36, ..., 24, 45, 14],
       [50, 57, 60, ..., 45, 46, 48]])

# Accuracy for Train Data:


In [53]:
# Accuracy:
clf_train_pred_F=clf_F.predict(X_train_transformed_F)
clf_acc_score_F = accuracy_score(y_F_Train, clf_train_pred_F)
print("Accuracy_score:",clf_acc_score_F)

train_F_f1 = f1_score(y_F_Train, clf_train_pred_F, average='weighted')
train_F_f2 = f1_score(y_F_Train, clf_train_pred_F, average='macro')
train_F_f3 = f1_score(y_F_Train, clf_train_pred_F, average='micro')
print("f1_score:Weighted:",train_F_f1)
print("f1_score:macro:",train_F_f2)
print("f1_score:micro:",train_F_f3)

Accuracy_score: 1.0
f1_score:Weighted: 1.0
f1_score:macro: 1.0
f1_score:micro: 1.0


# Accuracy for Test Data:


In [54]:
# Accuracy:
clf_test_pred_F=clf_F.predict(X_test_transformed_F)
clf_acc_score_test_F = accuracy_score(y_F_test, clf_test_pred_F)
print("Accuracy_score:",clf_acc_score_test_F)

# F1-Score:
test_F_f1 = f1_score(y_F_test, clf_test_pred_F, average='weighted')
test_F_f2 = f1_score(y_F_test, clf_test_pred_F, average='macro')
test_F_f3 = f1_score(y_F_test, clf_test_pred_F, average='micro')
print("f1_score:Weighted:",test_F_f1)
print("f1_score:macro:",test_F_f2)
print("f1_score:micro:",test_F_f3)

Accuracy_score: 0.4951923076923077
f1_score:Weighted: 0.4884001134001134
f1_score:macro: 0.48894348894348894
f1_score:micro: 0.4951923076923077


In [55]:
roc_auc_score(y_F_test, clf_F.predict_proba(X_test_transformed_F)[:,1])

0.48312528895053164

# For the Handwriting Data Set:

In [56]:
X_H,y_H= load_UCR_UEA_dataset('Handwriting', return_X_y=True)
X_H_Train,X_H_test,y_H_Train,y_H_test= train_test_split(X_H,y_H, test_size=0.5, random_state=0)
X_H_3d = from_nested_to_3d_numpy(X_H_Train)
X_H_3d.shape
X_H_test_3d= from_nested_to_3d_numpy(X_H_test)
X_H_test_3d.shape

print(X_H_3d.shape)
print(X_H_test_3d.shape)

(500, 3, 152)
(500, 3, 152)


In [57]:
X_train_transformed_H = transform_data(X_H_3d, 10, [3], 1)
X_test_transformed_H = transform_data(X_H_test_3d, 10, [3], 1)

filter of size  3 * 122 was created

filter of size  3 * 122 was created



In [58]:


clf_H= RandomForestClassifier(n_estimators=100,random_state=0)
clf_H.fit(X_train_transformed_H, y_H_Train)

RandomForestClassifier(random_state=0)

In [59]:
clf_H.score(X_train_transformed_H, y_H_Train)

1.0

In [60]:
clf_H.apply(X_train_transformed_H)

array([[ 95,  81,  82, ...,  85,  73,  86],
       [118,  94, 123, ..., 122, 136, 109],
       [376, 378, 379, ..., 374, 387, 332],
       ...,
       [ 86,  78,  77, ...,  69, 119,  57],
       [ 65,  45,  62, ...,  69,  42,  45],
       [128, 116, 129, ..., 115, 138, 117]])

# Accuracy for Train Data:

In [61]:
# Accuracy:
clf_train_pred_H=clf_H.predict(X_train_transformed_H)
clf_acc_score_H = accuracy_score(y_H_Train, clf_train_pred_H)
print("Accuracy_score:",clf_acc_score_H)

train_H_f1 = f1_score(y_H_Train, clf_train_pred_H, average='weighted')
train_H_f2 = f1_score(y_H_Train, clf_train_pred_H, average='macro')
train_H_f3 = f1_score(y_H_Train, clf_train_pred_H, average='micro')
print("f1_score:Weighted:",train_H_f1)
print("f1_score:macro:",train_H_f2)
print("f1_score:micro:",train_H_f3)

Accuracy_score: 1.0
f1_score:Weighted: 1.0
f1_score:macro: 1.0
f1_score:micro: 1.0


In [62]:
roc_auc_score(y_H_Train, clf_H.predict_proba(X_train_transformed_H), multi_class='ovr')

1.0

# Accuracy for Test Data:

In [63]:
# Accuracy:
clf_test_pred_H=clf_H.predict(X_test_transformed_H)
clf_acc_score_test_H = accuracy_score(y_H_test, clf_test_pred_H)
print("Accuracy_score:",clf_acc_score_test_H)

# F1-Score:
test_H_f1 = f1_score(y_H_test, clf_test_pred_H, average='weighted')
test_H_f2 = f1_score(y_H_test, clf_test_pred_H, average='macro')
test_H_f3 = f1_score(y_H_test, clf_test_pred_H, average='micro')
print("f1_score:Weighted:",test_H_f1)
print("f1_score:macro:",test_H_f2)
print("f1_score:micro:",test_H_f3)

Accuracy_score: 0.052
f1_score:Weighted: 0.049421875331133965
f1_score:macro: 0.04969125433543538
f1_score:micro: 0.052


In [64]:
roc_auc_score(y_H_test, clf_H.predict_proba(X_test_transformed_H), multi_class='ovr')

0.5380957279475314

# For Basic Motions:


In [65]:
X_B, X_B_test, y_B, y_B_test = load_basic_motions(return_X_y=True)


print(X_B.shape)
print(X_B_test.shape)

(40, 6, 100)
(40, 6, 100)


In [66]:
X_train_transformed_B = transform_data(X_B, 16, [3], 1)
X_test_transformed_B = transform_data(X_B_test, 16, [3], 1)

filter of size  6 * 52 was created

filter of size  6 * 52 was created



In [67]:


clf_B= RandomForestClassifier(n_estimators=100,random_state=0)
clf_B.fit(X_train_transformed_B, y_B)

RandomForestClassifier(random_state=0)

In [68]:
clf_B.score(X_train_transformed_B, y_B)

1.0

# Accuracy for Train Data:

In [69]:
# Accuracy:
clf_train_pred_B=clf_B.predict(X_train_transformed_B)
clf_acc_score_B = accuracy_score(y_B, clf_train_pred_B)
print("Accuracy_score:",clf_acc_score_B)

train_B_f1 = f1_score(y_B, clf_train_pred_B, average='weighted')
train_B_f2 = f1_score(y_B, clf_train_pred_B, average='macro')
train_B_f3 = f1_score(y_B, clf_train_pred_B, average='micro')
print("f1_score:Weighted:",train_B_f1)
print("f1_score:macro:",train_B_f2)
print("f1_score:micro:",train_B_f3)

Accuracy_score: 1.0
f1_score:Weighted: 1.0
f1_score:macro: 1.0
f1_score:micro: 1.0


In [70]:
roc_auc_score(y_B, clf_B.predict_proba(X_train_transformed_B), multi_class='ovr')

1.0

# Accuracy for Test Data:

In [71]:
# Accuracy:
clf_test_pred_B=clf_B.predict(X_test_transformed_B)
clf_acc_score_test_B = accuracy_score(y_B_test, clf_test_pred_B)
print("Accuracy_score:",clf_acc_score_test_B)

# F1-Score:
test_B_f1 = f1_score(y_B_test, clf_test_pred_B, average='weighted')
test_B_f2 = f1_score(y_B_test, clf_test_pred_B, average='macro')
test_B_f3 = f1_score(y_B_test, clf_test_pred_B, average='micro')
print("f1_score:Weighted:",test_B_f1)
print("f1_score:macro:",test_B_f2)
print("f1_score:micro:",test_B_f3)

Accuracy_score: 0.325
f1_score:Weighted: 0.2909751538783797
f1_score:macro: 0.2909751538783797
f1_score:micro: 0.325


In [72]:
roc_auc_score(y_B_test, clf_B.predict_proba(X_test_transformed_B), multi_class='ovr')

0.5833333333333334